In [3]:
!pip install langchain openai pinecone-client

In [23]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# Configurar las claves de API como variables de entorno
os.environ["OPENAI_API_KEY"] = "apikey"
os.environ["PINECONE_API_KEY"] = "apikey"

# Inicializar el cliente de Pinecone
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# Nombre del índice de Pinecone
index_name = "tallerunamagentes"

# Verificar si el índice ya existe
if index_name not in [index["name"] for index in pc.list_indexes()]:
    # Crear el índice si no existe
    pc.create_index(
        name=index_name,
        dimension=1536,  # Dimensión del modelo de embeddings utilizado
        metric="cosine"
    )

# Conectar al índice
index = pc.Index(index_name)

# Cargar el documento PDF
loader = PyPDFLoader("./manualUsuario.pdf")
documents = loader.load()

# Dividir el texto en fragmentos
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Inicializar el modelo de embeddings de OpenAI
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Crear la instancia de PineconeVectorStore y cargar los documentos
vectorstore = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

print("Documentos cargados exitosamente en Pinecone.")


Documentos cargados exitosamente en Pinecone.
